In [11]:
import pandas as pd
import random
from d20 import roll

In [12]:
excel_file = 'https://github.com/btaylor77/dnd/blob/main/beyond_pandas.xlsx?raw=true'
#excel_file = '/Users/Brad/Documents/git/dnd/beyond_pandas.xlsx'
hoard_df = pd.read_excel(excel_file,'hoard_items',engine='openpyxl').fillna(0)
spell_df = pd.read_excel(excel_file,'spell_list',index_col='Spell Name',engine='openpyxl')
class_df = pd.read_excel(excel_file,'class_spell_list',index_col='Spell Name',engine='openpyxl')
items_df = pd.read_excel(excel_file,'magic_items',index_col='item_name',engine='openpyxl')
coins_df = pd.read_excel(excel_file,'coins',engine='openpyxl').fillna(0)
treasure_df = pd.read_excel(excel_file,'treasure',engine='openpyxl').fillna(0)
damage_types = ['acid','cold','fire','force','lightning','necrotic','psychic','poison','radiant','thunder']
monster_types = ['Abberation','Beast','Celestial','Construct','Dragon','Elemental','Fey','Fiend','Giant','Humanoid','Monstrosity','Ooze','Plant','Undead']
swords = ['Greatsword','Longsword','Rapier','Scimitar','Shortsword']
weapons = swords + ['Battleaxe','Blowgun','Club','Crossbow, hand','Crossbow, heavy','Crossbow, light','Dagger','Dart','Flail','Glaive','Greataxe','Greatclub','Halberd','Handaxe','Javelin','Lance','Light hammer','Longbow','Mace','Maul','Morningstar','Net','Pike','Quarterstaff','Shortbow','Sickle','Sling','Spear','Trident','War pick','Warhammer','Whip',]

party = [('bard','base'),('bard','optional'),
          ('cleric','peace'),('cleric','base'),('cleric','optional'),
          ('warlock','base'),('warlock','fiend'),('warlock','optional'),
          ('wizard','base'),('wizard','optional')
        ]
all_class = [('artificer','base'),('bard','base'),('cleric','base'),('druid','base'),('paladin','base'),('ranger','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
arcane_class = [('artificer','base'),('bard','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
divine_class = [('cleric','base'),('druid','base'),('paladin','base'),('ranger','base')]


In [13]:
def roll_spells(min_level,max_level,classes,n=1):
    target_df = class_df.copy()
    target_df = target_df[(target_df['Level'] >= min_level) & (target_df['Level'] <= max_level) ]
    target_df = target_df[target_df[["Class","Subclass"]].apply(tuple, 1).isin(classes)]
    spells = target_df.index.values.tolist()
    select = pd.Series(random.choices(spells,k=n)).unique()
    target_df = spell_df.loc[select].sort_values(['Level','Spell Name'])
    return target_df

def roll_magic_items(table,n):
    if n > 0:
        target_df = items_df.copy()
        target_df = target_df[target_df['table_name']==table]
        target_df = target_df.reset_index()
        try:
            select = random.choices(target_df.index.values.tolist(),k=n,weights=list(target_df['weight']))
        except:
            print(table)
        target_df = target_df.loc[select]
        target_df['item_name'] = target_df.apply(lambda row: 'Scroll Of ' + roll_spells(row['spell_level'],row['spell_level'],party,n=1).index.values[0] if row['spell_level'] in range(9) else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(damage_types) + ')' if row['damage_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(monster_types) + ')' if row['monster_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(swords) + ')' if row['sword_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(weapons) + ')' if row['weapon_flag'] == 1 else row['item_name'], axis=1)
        target_df = target_df.sort_values('item_name')
    else:
        target_df = pd.DataFrame(columns=['item_name'])
    return target_df

def roll_coins(crs=[],coin_type='single'):
    result_df = pd.DataFrame()
    for cr in crs:
        target_df = coins_df.copy()
        target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr) & (target_df['coin_type']==coin_type)]
        select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
        target_df = target_df.loc[select][['CP','SP','EP','GP','PP']]
        result_df = result_df.append(target_df)
        result_df['CP'] = result_df['CP'].apply(lambda x: roll(str(x)).total)
        result_df['SP'] = result_df['SP'].apply(lambda x: roll(str(x)).total)
        result_df['EP'] = result_df['EP'].apply(lambda x: roll(str(x)).total)
        result_df['GP'] = result_df['GP'].apply(lambda x: roll(str(x)).total)
        result_df['PP'] = result_df['PP'].apply(lambda x: roll(str(x)).total)
        total_coins = pd.DataFrame(result_df.sum(axis=0)).transpose()
#        result_df = result_df.apply(lambda x: roll(x).total)
    result_df = total_coins.transpose().reset_index()
    result_df['treasure'] = result_df.apply(lambda x: str(x[0]) + ' ' + x['index'].lower(),axis=1)
    coins = result_df[result_df[0] > 0]['treasure'].values.tolist()
    return coins

def roll_treasure(gp,n):
    results = []
    if n > 0:
        target_df = treasure_df[treasure_df['GP']==gp]
        results = []
        for i in range(n):
            select = random.choice(target_df.index.values.tolist())
            results.append(select)
        target_df = target_df.loc[results].groupby('treasure_name').count()
        target_df = target_df.rename(columns={"GP":"count"})
        target_df['each'] = target_df.apply(lambda row: ' each' if row['count']>1 else '',axis=1)

        target_df = target_df.reset_index()
        target_df['treasure_desc'] = target_df.apply(lambda row: str(row['count']) + ' ' + row['treasure_name'] + ' worth ' + str(gp) + 'gp' + row['each'],axis=1)
        results = target_df['treasure_desc'].values.tolist()
    return results

def roll_hoard(cr):
    loot = []
    target_df = hoard_df.copy()
    target_df['cr'] = cr
    target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr)]
    select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
    target_df = target_df.loc[select]
    target_df['treasure_die'] = target_df['treasure_die'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_1'] = target_df['magic_item_die_1'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_2'] = target_df['magic_item_die_2'].apply(lambda x: roll(str(x)).total)
    target_df = target_df.drop(columns=['min_cr','max_cr','weight'])
    items1 = list(roll_magic_items(target_df['magic_item_table_1'].values[0],target_df['magic_item_die_1'].values[0])['item_name'].values.tolist())
    items2 = list(roll_magic_items(target_df['magic_item_table_2'].values[0],target_df['magic_item_die_2'].values[0])['item_name'].values.tolist())
    treasure = roll_treasure(target_df['treasure_value'].values[0],target_df['treasure_die'].values[0])
    coins = roll_coins([cr],'hoard')
    loot = loot + treasure + coins + items1 + items2
    target_df = target_df.reset_index(drop=True)

    return loot#target_df

In [14]:
for i in roll_hoard(15):
    print(i)

1 Bottle stopper cork embossed with gold leaf and set with amethysts worth 750gp
1 Carved harp of exotic wood with ivory inlay and zircon gems worth 750gp
1 Ceremonial electrum dagger with a black pearl in the pommel worth 750gp
2 Painted gold war mask worth 750gp each
19000 gp
2000 pp
All-Purpose Tool, +3
Armor, +1 Studded Leather


In [15]:
for i in roll_coins([5,5,5,3,3,2,3,5,10],'single'):
    print(i)

916 cp
137 sp
20 ep
340 gp
18 pp
